In [1]:
import os
import json
import wandb
import shutil
import numpy as np
from glob import glob
from tqdm import tqdm
from PIL import Image, ImageOps
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

from sahi.utils.file import save_json
from sahi.utils.coco import (
    Coco, CocoCategory, CocoImage, CocoAnnotation
)

In [2]:
train_annotation_file = "./artifacts/bdd100k-dataset:v0/labels/bdd100k_labels_images_train.json"
with open(train_annotation_file) as json_file:
    train_annotations = json.load(json_file)
len(train_annotations)

69863

In [3]:
count = 0
IMAGE_DIR = "./artifacts/bdd100k-dataset:v0/images/10k/train/"
for annotation in tqdm(train_annotations):
    image_file = annotation["name"]
    if os.path.isfile(os.path.join(IMAGE_DIR, image_file)):
        for label in annotation["labels"]:
            if "box2d" in label.keys():
                count += 1
                break
count

100%|██████████| 69863/69863 [00:00<00:00, 218815.90it/s]


2972

In [4]:
def corners_to_yolo(x1, y1, x2, y2, image_w, image_h):
    return [
        ((x2 + x1) / (2 * image_w)),
        ((y2 + y1) / (2 * image_h)),
        (x2 - x1) / image_w,
        (y2 - y1) / image_h
    ]

In [5]:
LABEL_MAP = {
    "person": 0,
    "rider": 1,
    "car": 2,
    "truck": 3,
    "bus": 4,
    "train": 5,
    "motor": 6,
    "bike": 7,
    "traffic light": 8,
    "traffic sign": 9,
}

In [6]:
all_10k_annotated_images = []
for annotation in tqdm(train_annotations):
    image_file = annotation["name"]
    if os.path.isfile(os.path.join(IMAGE_DIR, image_file)):
        all_10k_annotated_images.append(image_file)
        width, height = Image.open(os.path.join(IMAGE_DIR, image_file)).size
        current_labels = ""
        for label in annotation["labels"]:
            if "box2d" in label.keys():
                bbox = corners_to_yolo(
                    label["box2d"]["x1"],
                    label["box2d"]["y1"],
                    label["box2d"]["x2"],
                    label["box2d"]["y2"],
                    width, height
                )
                category_name = label["category"]
                category_id = LABEL_MAP[category_name]
                current_labels += " ".join(str(i) for i in [category_id] + bbox) + "\n"
        # print(current_labels)
        label_file_name = image_file.split(".")[0] + ".txt"
        with open(os.path.join(
            "./artifacts/bdd100k-dataset:v0/bdd100k-yolov5/labels", label_file_name), "w") as f:
            f.write(current_labels)

100%|██████████| 69863/69863 [00:01<00:00, 60542.24it/s]


In [7]:
all_annotations = glob("./artifacts/bdd100k-dataset:v0/bdd100k-yolov5/labels/*")
len(all_10k_annotated_images), len(all_annotations)

(2972, 2972)

In [8]:
for filename in tqdm(all_10k_annotated_images):
    shutil.copyfile(
        os.path.join(IMAGE_DIR, filename),
        os.path.join("./artifacts/bdd100k-dataset:v0/bdd100k-yolov5/images", filename)
    )

100%|██████████| 2972/2972 [00:00<00:00, 6472.50it/s]


In [9]:
all_images = glob("./artifacts/bdd100k-dataset:v0/bdd100k-yolov5/images/*")
len(all_images), len(all_annotations)

(2972, 2972)

In [10]:
train_images, val_images, train_labels, val_labels = train_test_split(
    sorted(all_images), sorted(all_annotations), test_size=0.3, random_state=42
)
val_images, test_images, val_labels, test_labels = train_test_split(
    sorted(val_images), sorted(val_labels), test_size=0.4, random_state=42
)

In [11]:
print(len(train_images), len(train_labels))
print(len(val_images), len(val_labels))
print(len(test_images), len(test_labels))

2080 2080
535 535
357 357


In [12]:
for idx in tqdm(range(len(train_images))):
    shutil.copyfile(
        train_images[idx],
        os.path.join(
            "./artifacts/bdd100k-dataset:v0/bdd100k-yolov5/train/images",
            train_images[idx].split("/")[-1]
        )
    )

    
for idx in tqdm(range(len(train_labels))):
    shutil.copyfile(
        train_labels[idx],
        os.path.join(
            "./artifacts/bdd100k-dataset:v0/bdd100k-yolov5/train/labels",
            train_labels[idx].split("/")[-1]
        )
    )

100%|██████████| 2080/2080 [00:00<00:00, 11975.16it/s]


In [13]:
for idx in tqdm(range(len(val_images))):
    shutil.copyfile(
        val_images[idx],
        os.path.join(
            "./artifacts/bdd100k-dataset:v0/bdd100k-yolov5/val/images",
            val_images[idx].split("/")[-1]
        )
    )

    
for idx in tqdm(range(len(val_labels))):
    shutil.copyfile(
        val_labels[idx],
        os.path.join(
            "./artifacts/bdd100k-dataset:v0/bdd100k-yolov5/val/labels",
            val_labels[idx].split("/")[-1]
        )
    )

100%|██████████| 535/535 [00:00<00:00, 11937.06it/s]


In [14]:
for idx in tqdm(range(len(test_images))):
    shutil.copyfile(
        test_images[idx],
        os.path.join(
            "./artifacts/bdd100k-dataset:v0/bdd100k-yolov5/test/images",
            test_images[idx].split("/")[-1]
        )
    )

    
for idx in tqdm(range(len(test_labels))):
    shutil.copyfile(
        test_labels[idx],
        os.path.join(
            "./artifacts/bdd100k-dataset:v0/bdd100k-yolov5/test/labels",
            test_labels[idx].split("/")[-1]
        )
    )

100%|██████████| 357/357 [00:00<00:00, 11999.28it/s]
